# **Personal Expense Tracker** - Project 1

**Student Name**: Carllos Watts-Nogueira

**Section**: 2504-FTB-CT-AIM-PT

**Due**: May 12 by 12:59am

**Academic Information**

Program: Artificial Intelligence and Machine Learning

Anticipated Graduation: October 2025

Institutions:

- University of San Diego, San Diego – CA, United States
- Fullstack Academy, Brooklyn – NY, USA

Key Topics Covered:

- Applied Data Science; - Machine Learning; - Deep Learning; - Natural Language Processing (NLP); - Generative AI; 

Program Focus:

Hands-on skill development for AI and Machine Learning Engineering, emphasizing real-world projects and technical mastery.







In [ ]:
# ---------------- Import ----------------
import csv #This module is used to read from and write to CSV files. (expenses and budget CSV)
from datetime import datetime, date # this statment imports the datetime and date classes from the datetime module. date.today() returns the current date.

# ---------------- Global Variables ----------------
# Lists (Dictonary)
# both list expenses and budgets hold records in memory (during the section)
# Each expense or budget record is stored as a dictionary with its related fields.
expenses = []       # List, Cada despesa é um dicionário com: id, date, category, amount, description.
budgets = []        # List, Cada orçamento é um dicionário com: id, month, year, budget_amount.

#Id Counters, for each new record gets a unique id.
next_expense_id = 1 #
next_budget_id = 1 #

#Files names, names of the csv files for save and load functions rely on these names.
EXPENSES_FILE = 'personalexpenses.csv'
BUDGET_FILE = 'budget.csv'

# Dictionary, Add Expenses Category picklist, for easy input for the final user, adding number related the caterogy name.
# Dicionário de categorias permitidas (usado como picklist)
allowed_categories = {
    "1": "Food",
    "2": "Travel",
    "3": "Entertainment",
    "4": "Groceries",
    "5": "Health",
    "6": "Utilities",
    "7": "Education",
    "8": "Shopping"
}

# ---------------- Add Expense Function ----------------
def add_expense():
    """
    Adiciona uma nova despesa solicitando os campos um a um.
    Os campos obrigatórios são: date (YYYY-MM-DD), category e amount.
    A descrição é opcional.
    A despesa é armazenada em memória; os dados são salvos no arquivo apenas via a opção "Save Data".
    """
    global next_expense_id
    while True:
        date_input = input("Enter date (YYYY-MM-DD): ").strip()
        if not date_input:
            print("Date is required. Please try again.")
            continue
        try:
            exp_date = datetime.strptime(date_input, "%Y-%m-%d")
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD.")
            continue
        break
    print("Allowed Categories:")
    for code, cat in allowed_categories.items():
        print(f"[{code}] {cat}", end="  ")
    print()
    while True:
        cat_choice = input("Choose category by code: ").strip()
        if not cat_choice:
            print("Category is required. Please try again.")
            continue
        if cat_choice not in allowed_categories:
            print("Invalid category code. Please choose from the list.")
            continue
        category = allowed_categories[cat_choice]
        break
    while True:
        amount_input = input("Enter amount: ").strip()
        if not amount_input:
            print("Amount is required. Please try again.")
            continue
        try:
            amount = float(amount_input)
        except ValueError:
            print("Invalid amount, please enter a numeric value.")
            continue
        break

    description = input("Enter description (optional): ").strip()

    expense = {
        'id': next_expense_id,
        'date': date_input,
        'category': category,
        'amount': amount,
        'description': description
    }
    next_expense_id += 1
    expenses.append(expense)
    print("Expense added successfully!")

    # Exibe o status do orçamento para o período da despesa (se houver orçamento definido)
    exp_month = exp_date.strftime("%m")
    exp_year = exp_date.strftime("%Y")
    budget_record = next((b for b in budgets if b.get('month') == exp_month and b.get('year') == exp_year), None)
    if budget_record:
        total_for_month = sum(e['amount'] for e in expenses if e['date'].startswith(f"{exp_year}-{exp_month}"))
        remaining = float(budget_record['budget_amount']) - total_for_month
        print(f"Budget for {exp_year}-{exp_month}: ${float(budget_record['budget_amount']):.2f}")
        print(f"Total Expenses for this period: ${total_for_month:.2f}")
        if remaining < 0:
            print(f"Exceeded: ${abs(remaining):.2f}")
            print(f"For {exp_year}-{exp_month} you have exceeded your budget by ${abs(remaining):.2f}.")
        else:
            print(f"Remaining: ${remaining:.2f}")
            print(f"For {exp_year}-{exp_month} you have ${remaining:.2f} remaining in your budget.")
    else:
        print(f"No budget set for {exp_year}-{exp_month}.")

# ---------------- Edit - Expense Functions ----------------
def edit_expense():
    """Permite editar uma despesa existente, identificada pelo ID informado."""
    if not expenses:
        print("No expenses available to edit.")
        return
    try:
        exp_id = int(input("Enter the ID of the expense you wish to edit: "))
    except ValueError:
        print("Invalid ID input.")
        return

    expense = next((e for e in expenses if e['id'] == exp_id), None)
    if not expense:
        print("Expense with that ID not found.")
        return

    print("Current expense details:")
    print(f"1. Date: {expense['date']}")
    print(f"2. Category: {expense['category']}")
    print(f"3. Amount: {expense['amount']}")
    print(f"4. Description: {expense['description']}")
    print("Press Enter to leave a field unchanged.")

    new_date = input("New Date (YYYY-MM-DD): ").strip()
    if new_date:
        try:
            datetime.strptime(new_date, "%Y-%m-%d")
            expense['date'] = new_date
        except ValueError:
            print("Invalid date format. Keeping previous value.")

    print("Allowed Categories:")
    for code, cat in allowed_categories.items():
        print(f"[{code}] {cat}", end="  ")
    print()
    new_cat = input("New Category code: ").strip()
    if new_cat:
        if new_cat in allowed_categories:
            expense['category'] = allowed_categories[new_cat]
        else:
            print("Invalid category code. Keeping previous value.")

    new_amount = input("New Amount: ").strip()
    if new_amount:
        try:
            expense['amount'] = float(new_amount)
        except ValueError:
            print("Invalid amount. Keeping previous value.")

    new_desc = input("New Description: ").strip()
    if new_desc:
        expense['description'] = new_desc

    print("Expense updated successfully!")

# ---------------- View - Expense Functions ----------------
def view_expenses():
    """
    Exibe as despesas com as seguintes opções:
      1. View All
      2. View Current Month
      3. View Specific Month
      4. View Incomplete Data (registros com data, category ou amount ausentes)
      5. Back to Main Menu
    """
    if not expenses:
        print("No expenses recorded yet.")
        return
    while True:
        print("\nView expenses options:")
        print("1. View All")
        print("2. View Current Month")
        print("3. View Specific Month")
        print("4. View Incomplete Data")
        print("5. Back to Main Menu")
        option = input("Select an option: ").strip()

        if option in {"1", "2", "3", "4", "5"}:
            break
        else:
            print("Invalid option. Please enter 1, 2, 3, 4, or 5. ")
    if option == "5":
        return
    filtered_expenses = []
    if option == "1":
        filtered_expenses = expenses
    elif option == "2":
        today = date.today()
        m = today.strftime("%m")
        y = today.strftime("%Y")
        filtered_expenses = [e for e in expenses if e['date'].startswith(f"{y}-{m}")]
    elif option == "3":
        while True:
            month_input = input("Enter month (MM): ").strip()
            if not month_input:
                print("Month cannot be empty. Please try again.")
                continue
            if not (month_input.isdigit() and len(month_input) == 2):
                print("Invalid month. Please enter a valid month in XX format (2 digits).")
                continue
            if not (1 <= int(month_input) <= 12):
                print("Invalid month. Please enter a valid month between 01 and 12.")
                continue
            month = month_input.zfill(2)
            break

        while True:
            year_input = input("Enter year (YYYY): ").strip()
            if not year_input:
                print("Year cannot be empty. Please try again.")
                continue
            if not (year_input.isdigit() and len(year_input) == 4):
                print("Invalid year. Please enter a valid year in YYYY format (4 digits).")
                continue
            year = year_input
            break
        filtered_expenses = [e for e in expenses if e['date'].startswith(f"{year}-{month}")]
        if not filtered_expenses:
            print(f"No expense records found for {year}-{month}.")
            return
    elif option == "4":
      filtered_expenses = [e for e in expenses if (not e.get('date') or not e.get('category') or str(e.get('amount')).strip() == "" or str(e.get('description', '')).strip() == "")]
      if not filtered_expenses:
          print("No incomplete expense records found.")
          return
    else:
        print("Invalid option. Showing all expenses.")
        filtered_expenses = expenses

    try:
        filtered_expenses.sort(key=lambda x: datetime.strptime(x['date'], "%Y-%m-%d") if x.get('date') else datetime.min)
    except Exception as ex:
        print("Error sorting expenses by date:", ex)

    print("\n{:<5} {:<12} {:<15} {:<10} {:<40}".format("ID", "Date", "Category", "Amount", "Description"))
    print("-" * 85)
    for exp in filtered_expenses:
        print("{:<5} {:<12} {:<15} {:<10.2f} {:<40}".format(exp['id'],
                                                              exp.get('date', ''),
                                                              exp.get('category', ''),
                                                              exp.get('amount', 0),
                                                              exp.get('description', '')))

# ---------------- Add/Edit Budget Functions ----------------
def set_budget():
    """
    Adiciona ou edita o orçamento para um determinado mês/ano.
    Se já houver um orçamento para o mesmo período, permite apenas a edição.
    Os dados são armazenados em memória e somente salvos no arquivo via "Save Data".
    """
    global next_budget_id

    while True:
        month_input = input("Enter budget month (MM): ").strip()
        if not month_input:
            print("Month is required. Please try again.")
            continue
        if not (month_input.isdigit() and len(month_input) == 2):
            print("Invalid month. Please enter a valid month in MM format (e.g., 01, 02, ..., 12).")
            continue
        if not (1 <= int(month_input) <= 12):
            print("Invalid month. Please enter a valid month between 01 and 12.")
            continue

        month = month_input.zfill(2)
        break

    while True:
        year_input = input("Enter budget year (YYYY): ").strip()
        if not year_input:
            print("Year is required. Please try again.")
            continue
        if not (year_input.isdigit() and len(year_input) == 4):
            print("Invalid year. Please enter a valid 4-digit year (e.g., 2024).")
            continue
        year = year_input
        break

    while True:
        budget_input = input("Enter your budget for the month: ").strip()
        if not budget_input:
            print("Budget amount is requered. Please try again.")
        try:
            budget_amount = float(budget_input)
            break
        except ValueError:
            print("Invalid amount entered.")
            continue

    record = next((b for b in budgets if b.get('month') == month and b.get('year') == year), None)
    if record:
        print(f"Budget for {year}-{month} already exists with amount ${record['budget_amount']:.2f}.")
        choice = input("Do you want to update it? (y/n): ").lower().strip()
        if choice == 'y':
            record['budget_amount'] = budget_amount
            print("Budget updated successfully.")
        else:
            print("Budget not modified.")
    else:
        new_budget = {
            'id': next_budget_id,
            'month': month,
            'year': year,
            'budget_amount': budget_amount
        }
        next_budget_id += 1
        budgets.append(new_budget)
        print("Budget record added successfully.")

# ---------------- Track Budget Functions ----------------
def track_budget():
    """
    Permite visualizar as informações de orçamento para:
      1. Yearly
      2. Current Month
      3. Specific Month
      4. Months Budget Exceeded (lista de meses com gastos acima do orçamento)
      5. Months Below Budget (lista de meses com gastos menores que o orçamento)
      6. Back to Main Menu
    Exibe, para cada opção, Budget, Total Expenses, Remaining ou Exceeded e a mensagem final.
    """
    while True:
        print("\nTrack Budget Options:")
        print("1. Yearly")
        print("2. Current Month")
        print("3. Specific Month")
        print("4. Months Budget Exceeded")
        print("5. Months Below Budget")
        print("6. Back to Main Menu")
        option = input("Select an option: ").strip()

        if option in {"1", "2", "3", "4", "5", "6"}:
            break
        else:
            print("Invalid option. Please enter 1, 2, 3, 4, 5, or 6. ")
    if option == "6":
        return

    if option == "1":
        while True:
            year_input = input("Enter the year (YYYY): ").strip()
            if not year_input:
                print("Year cannot be empty.")
                continue
            if not (year_input.isdigit() and len(year_input) == 4):
                print("Invalid year. Please enter a valid 4-digit year (e.g., 2024).")
                continue
            year = year_input
            break

        total_budget = sum(b['budget_amount'] for b in budgets if b.get('year') == year)
        total_expenses = sum(e['amount'] for e in expenses if e['date'].startswith(f"{year}-"))
        remaining = total_budget - total_expenses
        print(f"\nFor Year {year}:")
        print("Budget: $" + f"{total_budget:.2f}")
        print("Total Expenses: $" + f"{total_expenses:.2f}")
        if remaining < 0:
            print("Exceeded: $" + f"{abs(remaining):.2f}")
            print(f"You have exceeded your budget for the year by ${abs(remaining):.2f}.")
        else:
            print("Remaining: $" + f"{remaining:.2f}")
            print(f"You have ${remaining:.2f} remaining in your budget for the year.")

    elif option == "2":
        today = date.today()
        m = today.strftime("%m")
        y = today.strftime("%Y")
        budget_record = next((b for b in budgets if b.get('month') == m and b.get('year') == y), None)
        total_budget = float(budget_record['budget_amount']) if budget_record else 0.0
        total_expenses = sum(e['amount'] for e in expenses if e['date'].startswith(f"{y}-{m}"))
        remaining = total_budget - total_expenses
        print(f"\nFor Current Month ({y}-{m}):")
        print("Budget: $" + f"{total_budget:.2f}")
        print("Total Expenses: $" + f"{total_expenses:.2f}")
        if remaining < 0:
            print("Exceeded: $" + f"{abs(remaining):.2f}")
            print(f"You have exceeded your budget for this month by ${abs(remaining):.2f}.")
        else:
            print("Remaining: $" + f"{remaining:.2f}")
            print(f"You have ${remaining:.2f} remaining in your budget for this month.")

    elif option == "3":
        while True:
            month_input = input("Enter month (MM): ").strip()
            if not month_input:
                print("Month cannot be empty.")
                continue
            if not (month_input.isdigit() and len(month_input) == 2):
                print("Invalid month. Please enter the month in MM format (2 digits, e.g., 01, 02, ..., 12).")
                continue
            if not (1 <= int(month_input) <= 12):
                print("Invalid month. Please enter a valid month between 01 and 12.")
                continue
            month = month_input.zfill(2)
            break
        while True:
            year_input = input("Enter year (YYYY): ").strip()
            if not year_input:
                print("Year cannot be empty.")
                continue
            if not (year_input.isdigit() and len(year_input) == 4):
                print("Invalid year. Please enter the year in YYYY format (4 digits, e.g., 2024).")
                continue
            year = year_input
            break
        budget_record = next((b for b in budgets if b.get('month') == month and b.get('year') == year), None)
        total_budget = float(budget_record['budget_amount']) if budget_record else 0.0
        total_expenses = sum(e['amount'] for e in expenses if e['date'].startswith(f"{year}-{month}"))
        remaining = total_budget - total_expenses
        print(f"\nFor Month {year}-{month}:")
        print("Budget: $" + f"{total_budget:.2f}")
        print("Total Expenses: $" + f"{total_expenses:.2f}")
        if remaining < 0:
            print("Exceeded: $" + f"{abs(remaining):.2f}")
            print(f"You have exceeded your budget for {year}-{month} by ${abs(remaining):.2f}.")
        else:
            print("Remaining: $" + f"{remaining:.2f}")
            print(f"You have ${remaining:.2f} remaining in your budget for {year}-{month}.")

    elif option == "4":
        # Listar meses (a partir dos registros de orçamento) em que o total de despesas foi maior que o orçamento.
        found = False
        print("\nMonths Budget Exceeded:")
        # Ordena os registros por ano e mês.
        for b in sorted(budgets, key=lambda x: (x['year'], x['month'])):
            mon = b['month']
            yr = b['year']
            total_budget = float(b['budget_amount'])
            total_expenses = sum(e['amount'] for e in expenses if e['date'].startswith(f"{yr}-{mon}"))
            if total_expenses > total_budget:
                exceeded = total_expenses - total_budget
                print(f"For {yr}-{mon}: Budget: ${total_budget:.2f}, Total Expenses: ${total_expenses:.2f}.")
                print(f"You have exceeded your budget for this month by ${exceeded:.2f}.")
                found = True
        if not found:
            print("No months found where the expenses exceeded the budget.")

    elif option == "5":
        # Listar meses em que o total de despesas ficou abaixo do orçamento.
        found = False
        print("\nMonths Below Budget:")
        for b in sorted(budgets, key=lambda x: (x['year'], x['month'])):
            mon = b['month']
            yr = b['year']
            total_budget = float(b['budget_amount'])
            total_expenses = sum(e['amount'] for e in expenses if e['date'].startswith(f"{yr}-{mon}"))
            if total_expenses < total_budget:
                remaining = total_budget - total_expenses
                print(f"For {yr}-{mon}: Budget: ${total_budget:.2f}, Total Expenses: ${total_expenses:.2f}.")
                print(f"You have ${remaining:.2f} remaining in your budget for this month.")
                found = True
        if not found:
            print("No months found where there is remaining budget (or all are exactly met).")

    else:
        print("Invalid option.")
# ---------------- View Budget Functions ----------------
def view_budget():
    """
    Exibe os registros de orçamento com as seguintes opções:
      1. View All Budgets
      2. View Budget for Current Month
      3. View Budget for Specific Month
      4. Back to Main Menu
    """
    while True:
        print("\nView Budget Options:")
        print("1. View All Budgets")
        print("2. View Budget for Current Month")
        print("3. View Budget for Specific Month")
        print("4. Back to Main Menu")
        option = input("Select an option: ").strip()

        if option in {"1","2","3","4"}:
            break
        else:
            print("Invalid option. Please enter 1,2,3, or 4. ")
    if option == "4":
        return
    if option == "1":
        if not budgets:
            print("No budget records found.")
        else:
            budgets_sorted = sorted(budgets, key=lambda b: (b.get('year'), b.get('month')))
            print("\n{:<5} {:<8} {:<6} {:<12}".format("ID", "Month", "Year", "Budget Amount"))
            print("-" * 40)
            for b in budgets_sorted:
                print("{:<5} {:<8} {:<6} ${:<12.2f}".format(b.get("id", ""),
                                                             b.get("month", ""),
                                                             b.get("year", ""),
                                                             float(b.get("budget_amount", 0))))
    elif option == "2":
        today = date.today()
        m = today.strftime("%m")
        y = today.strftime("%Y")
        budget_record = next((b for b in budgets if b.get('month') == m and b.get('year') == y), None)
        if not budget_record:
            print("No budget found for the current month.")
        else:
            print(f"Budget for current month ({y}-{m}): ${float(budget_record.get('budget_amount', 0)):.2f}")
    elif option == "3":
        while True:
            month_input = input("Enter month (MM): ").strip()
            if not month_input:
                print("Month cannot be empty. Please try again.")
                continue
            if not (month_input.isdigit() and len(month_input) == 2):
                print("Invalid month. Please enter a 2-digit month (e.g., 01, 02, ..., 12).")
                continue
            month = month_input.zfill(2)
            break
        while True:
            year_input = input("Enter year (YYYY): ").strip()
            if not year_input:
                print("Year cannot be empty.")
                continue
            if not (year_input.isdigit() and len(year_input) == 4):
                print("Invalid year. Please enter a 4-digit year (e.g., 2024).")
                continue
            year = year_input
            break
        budget_record = next((b for b in budgets if b.get('month') == month and b.get('year') == year), None)
        if not budget_record:
            print(f"No budget found for {year}-{month}.")
        else:
            print(f"Budget for {year}-{month}: ${float(budget_record.get('budget_amount', 0)):.2f}")
    else:
        print("Invalid option.")

# ---------------- File Handling Functions ----------------
def save_expenses():
    """
    Salva as despesas no arquivo CSV utilizando encoding 'utf-8-sig'
    e newline='' para maior compatibilidade.
    """
    with open(EXPENSES_FILE, mode='w', encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['id', 'date', 'category', 'amount', 'description'])
        for exp in expenses:
            writer.writerow([exp['id'], exp['date'], exp['category'], exp['amount'], exp['description']])
    print("Expenses saved successfully!")

def load_expenses():
    """
    Carrega as despesas do arquivo CSV utilizando encoding 'utf-8-sig'
    e newline='' e atualiza o contador next_expense_id.
    """
    global expenses, next_expense_id
    try:
        with open(EXPENSES_FILE, mode='r', encoding='utf-8-sig', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            expenses = []
            max_id = 0
            for row in reader:
                row['id'] = int(row['id'])
                row['amount'] = float(row['amount'])
                expenses.append(row)
                if row['id'] > max_id:
                    max_id = row['id']
            next_expense_id = max_id + 1
        print("Expenses loaded successfully!")
    except FileNotFoundError:
        print("No saved expenses found. Starting fresh.")

def save_budgets():
    """
    Salva os registros de orçamento no arquivo CSV utilizando encoding 'utf-8-sig'
    e newline=''.
    """
    with open(BUDGET_FILE, mode='w', encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['id', 'month', 'year', 'budget_amount'])
        for b in budgets:
            writer.writerow([b['id'], b['month'], b['year'], b['budget_amount']])
    print("Budgets saved successfully!")

def load_budgets():
    """
    Carrega os registros de orçamento do arquivo CSV utilizando encoding 'utf-8-sig'
    e newline='', e atualiza o contador next_budget_id.
    """
    global budgets, next_budget_id
    try:
        with open(BUDGET_FILE, mode='r', encoding='utf-8-sig', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            budgets = []
            max_id = 0
            for row in reader:
                row['id'] = int(row['id'])
                row['budget_amount'] = float(row['budget_amount'])
                budgets.append(row)
                if row['id'] > max_id:
                    max_id = row['id']
            next_budget_id = max_id + 1
        print("Budgets loaded successfully!")
    except FileNotFoundError:
        print("No budget data found. Starting with empty budgets.")

# ---------------- Main Menu ----------------
def main():
    # Carrega os registros salvos (se existirem) no início.
    load_expenses()
    load_budgets()

    while True:
        print("\nPersonal Expense Tracker")
        print("1. Add Expense")
        print("2. Edit Expense")
        print("3. View Expenses")
        print("4. Add/Edit Budget")
        print("5. Track Budget")
        print("6. View Budget")
        print("7. Save Data")
        print("8. Exit")

        choice = input("Choose an option: ").strip()
        if choice == "1":
            add_expense()
        elif choice == "2":
            edit_expense()
        elif choice == "3":
            view_expenses()
        elif choice == "4":
            set_budget()
        elif choice == "5":
            track_budget()
        elif choice == "6":
            view_budget()
        elif choice == "7":
            save_expenses()
            save_budgets()
            print("Data saved successfully!")
        elif choice == "8":
            save_choice = input("Do you want to save data before exiting? (y/n): ").lower().strip()
            if save_choice == 'y':
                save_expenses()
                save_budgets()
            print("Exiting program.")
            break
        else:
            print("Invalid choice, please try again.")

if __name__ == "__main__":
    main()

No saved expenses found. Starting fresh.
No budget data found. Starting with empty budgets.

Personal Expense Tracker
1. Add Expense
2. Edit Expense
3. View Expenses
4. Add/Edit Budget
5. Track Budget
6. View Budget
7. Save Data
8. Exit
Allowed Categories:
[1] Food  [2] Travel  [3] Entertainment  [4] Groceries  [5] Health  [6] Utilities  [7] Education  [8] Shopping  
Expense added successfully!
No budget set for 2025-05.

Personal Expense Tracker
1. Add Expense
2. Edit Expense
3. View Expenses
4. Add/Edit Budget
5. Track Budget
6. View Budget
7. Save Data
8. Exit
Expenses saved successfully!
Budgets saved successfully!
Data saved successfully!

Personal Expense Tracker
1. Add Expense
2. Edit Expense
3. View Expenses
4. Add/Edit Budget
5. Track Budget
6. View Budget
7. Save Data
8. Exit
